In [ ]:
# 1. Create and activate a new virtual environment
python -m venv tf-env
tf-env\Scripts\activate              # .\tf-env\Scripts\activate on PowerShell

# 2. Upgrade packaging tools
pip install --upgrade pip wheel setuptools

# 3. Install a wheel that matches Windows 10 + Python 3.10
pip install "tensorflow==2.15.0"     # or 2.12.0 if your CPU lacks AVX
                                     
# 4. Verify import
python - << "PY"
import tensorflow as tf, platform, sys
print("TF:", tf.__version__, "Python:", sys.version)
PY


In [4]:
!python -m pip install mediapipe



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import sys
print(sys.executable)


c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python310\python.exe


In [22]:
import sys, subprocess

# call pip with the interpreter that is currently running this notebook
subprocess.check_call(
    [sys.executable, "-m", "pip", "uninstall", "-y",
     "tensorflow==2.10.0", "mediapipe==0.10.9"]
)


0

In [ ]:
!"c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python310\python.exe" -m pip uninstall tensorflow==2.10.0 mediapipe==0.10.9


In [20]:
!"c:\Users\Pranav Abani\AppData\Local\Programs\Python\Python310\python.exe" -m pip uninstall  "tensorflow>=2.15"



^C


In [19]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf

class FastEmotionDetector:
    def __init__(self):
        # MediaPipe face detection (ultra-fast)
        self.mp_face_detection = mp.solutions.face_detection
        self.face_detection = self.mp_face_detection.FaceDetection(
            model_selection=0, min_detection_confidence=0.5)
        
        # Load TensorFlow Lite emotion model (ultra-lightweight)
        self.interpreter = tf.lite.Interpreter(model_path="emotion_model.tflite")
        self.interpreter.allocate_tensors()
        
        # Emotion labels for your use case
        self.emotions = ['sad', 'happy', 'angry', 'neutral', 'confident']
    
    def detect_emotions(self, frame):
        # Convert BGR to RGB for MediaPipe
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.face_detection.process(rgb_frame)
        
        if results.detections:
            for detection in results.detections:
                # Get bounding box
                bbox = detection.location_data.relative_bounding_box
                h, w, _ = frame.shape
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)
                
                # Extract face region
                face = frame[y:y+height, x:x+width]
                
                # Predict emotion (optimized preprocessing)
                emotion = self.predict_emotion(face)
                
                # Draw results
                cv2.rectangle(frame, (x, y), (x+width, y+height), (0, 255, 0), 2)
                cv2.putText(frame, emotion, (x, y-10), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        return frame
    
    def predict_emotion(self, face_img):
        # Ultra-fast preprocessing
        face_img = cv2.resize(face_img, (48, 48))
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        face_img = face_img.astype('float32') / 255.0
        face_img = np.expand_dims(face_img, axis=[0, -1])
        
        # TensorFlow Lite inference (fastest)
        input_details = self.interpreter.get_input_details()
        output_details = self.interpreter.get_output_details()
        
        self.interpreter.set_tensor(input_details[0]['index'], face_img)
        self.interpreter.invoke()
        
        output = self.interpreter.get_tensor(output_details[0]['index'])
        emotion_idx = np.argmax(output)
        
        return self.emotions[emotion_idx]

# Usage for live video
def run_fast_detection():
    detector = FastEmotionDetector()
    cap = cv2.VideoCapture(0)  # Your video source
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
            
        # Process frame (optimized)
        result_frame = detector.detect_emotions(frame)
        
        cv2.imshow('Fast Emotion Detection', result_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()


ImportError: DLL load failed while importing _pywrap_tf2: The specified module could not be found.

In [ ]:
import tensorflow as tf
print(tf.__version__)


ModuleNotFoundError: No module named 'tensorflow'